In [2]:
from tensorflow.keras.models import load_model

import pandas as pd
import numpy as np

In [3]:
MODEL_DIR = './models/'

In [15]:
name = input('Enter model name: ')

model = load_model(MODEL_DIR + name + '_keras')
print('Loaded model: ' + name)
model

Loaded model: seq_200k


In [17]:
# Predict matchup
off_id = 203954
def_id = 203920

# Get offense stats
data = pd.read_csv('./stats/2021-22_off_stats.csv')
data = data.add_prefix('OFF_')
data.rename(columns={'OFF_SEASON_ID': 'SEASON_ID'}, inplace=True)
inputs_o = data[data['OFF_PLAYER_ID'] == off_id]

# Get defensive stats
def_data = pd.read_csv('./stats/2021-22_def_stats.csv')
def_data = def_data.fillna(0)
def_data = def_data.add_prefix('DEF_')
def_data.rename(columns={'DEF_SEASON_ID': 'SEASON_ID', 'DEF_DEF_RATING': 'DEF_RATING', 
                     'DEF_DEF_BOXOUTS': 'DEF_BOXOUTS'}, inplace=True)
inputs_d = def_data[def_data['DEF_PLAYER_ID'] == def_id]
print('Offensive player: ' + def_data[def_data['DEF_PLAYER_ID'] == off_id].reset_index()['DEF_PLAYER_NAME'][0])
print('Devensive player: ' + def_data[def_data['DEF_PLAYER_ID'] == def_id].reset_index()['DEF_PLAYER_NAME'][0])

# Get both inputs together
inputs_o.drop(columns=['OFF_PLAYER_ID', 'OFF_PTS_PER_100'], inplace=True)
inputs_d.drop(columns=['DEF_PLAYER_ID', 'DEF_PLAYER_NAME', 'SEASON_ID'], inplace=True)
inputs_o.drop(columns=['OFF_POST_TOUCH_PTS', 'OFF_POST_TOUCH_FG_PCT', 'OFF_FGM', 'OFF_EFF_FG_PCT', 'OFF_OREB', 'OFF_PAINT_TOUCH_FGA', 'OFF_PAINT_TOUCH_FGM', 'OFF_PAINT_TOUCH_TOV', 'OFF_PAINT_TOUCH_PASSES', 'OFF_PAINT_TOUCH_PTS', 'OFF_DREB', 'OFF_FG3M', 'OFF_FTM'], inplace=True)
inputs_d.drop(columns=['DEF_D_FGM', 'DEF_OPP_PTS_PAINT', 'DEF_CONTESTED_SHOTS_2PT', 'DEF_DEFLECTIONS', 'DEF_OPP_PTS_FB', 'DEF_OPP_PTS_2ND_CHANCE', 'DEF_OPP_PTS_OFF_TOV', 'DEF_PCT_PLUSMINUS', 'DEF_G'], inplace=True)

inputs_o = inputs_o.reset_index(drop=True)
inputs_d = inputs_d.reset_index(drop=True)
X_predict = pd.concat([inputs_d, inputs_o], axis=1)

X_predict.sort_index(axis=1, inplace=True)

# Normalize inputs
X_seqtest = pd.read_csv('./stats/X.csv')
X_seqtest.drop(columns=['OFF_POST_TOUCH_PTS', 'OFF_POST_TOUCH_FG_PCT', 'OFF_PTS_PER_100'], inplace=True)
X_seqtest.drop(columns=['OFF_MIN', 'OFF_GP', 'DEF_MIN', 'DEF_GP'], inplace=True)
# X_seqtest = X_seqtest[X_seqtest['SEASON_ID'] == 22021]

X_predict = (X_predict - X_seqtest.mean()) / X_seqtest.std()

X_predict = X_predict.fillna(0)

X_predict = X_predict.sort_index(axis=1)

print(X_predict.columns)

# X_predict.columns
y = model.predict(X_predict.to_numpy())

# Convert from per 100 to predicted per game
off_data = data[data['OFF_PLAYER_ID'] == off_id].reset_index()
poss = (off_data['OFF_PTS'] / (off_data['OFF_PTS_PER_100'] / 100))[0]

y[0][0] * poss / 100

Offensive player: Joel Embiid
Devensive player: Khem Birch
Index(['DEF_AGE', 'DEF_BLK', 'DEF_BOXOUTS', 'DEF_CHARGES_DRAWN',
       'DEF_CONTESTED_SHOTS', 'DEF_CONTESTED_SHOTS_3PT', 'DEF_DEF_WS',
       'DEF_DREB', 'DEF_DREB_PCT', 'DEF_D_FGA', 'DEF_D_FG_PCT', 'DEF_GP',
       'DEF_L', 'DEF_MIN', 'DEF_NORMAL_FG_PCT', 'DEF_PCT_BLK',
       'DEF_PCT_BOX_OUTS_REB', 'DEF_PCT_STL', 'DEF_PLAYER_HEIGHT_INCHES',
       'DEF_PLAYER_WEIGHT', 'DEF_RATING', 'DEF_STL', 'DEF_W', 'OFF_AGE',
       'OFF_AST', 'OFF_BLKA', 'OFF_CATCH_SHOOT_FG_PCT', 'OFF_CATCH_SHOOT_PTS',
       'OFF_DRIVE_FG_PCT', 'OFF_DRIVE_PTS', 'OFF_ELBOW_TOUCH_FG_PCT',
       'OFF_ELBOW_TOUCH_PTS', 'OFF_FG3A', 'OFF_FG3_PCT', 'OFF_FGA',
       'OFF_FG_PCT', 'OFF_FTA', 'OFF_FT_PCT', 'OFF_GP', 'OFF_MIN',
       'OFF_PAINT_TOUCHES', 'OFF_PAINT_TOUCH_FG_PCT', 'OFF_PF',
       'OFF_PLAYER_HEIGHT_INCHES', 'OFF_PLAYER_WEIGHT', 'OFF_PTS',
       'OFF_PULL_UP_FG_PCT', 'OFF_PULL_UP_PTS', 'OFF_REB', 'OFF_TOUCHES',
       'OFF_TOV', 'SEASON_ID'],


C:\Users\Darrick\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


ValueError: in user code:

    File "C:\Users\Darrick\anaconda3\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Darrick\anaconda3\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Darrick\anaconda3\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Darrick\anaconda3\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "C:\Users\Darrick\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Darrick\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 263, in assert_input_compatibility
        raise ValueError(f'Input {

In [131]:
27.383924089036544*22.3/62.6 + 26.606356474730347*22.3/62.6 + 28.225678796166772*18/62.6

27.34896924765181

In [20]:
loc = './stats/'
from sklearn.metrics import mean_absolute_error
from math import sqrt

X = pd.read_csv(loc + 'X.csv')
y = pd.read_csv(loc + 'y.csv')

# y = y[['PLAYER_PPG', 'PLAYER_APG', 'PLAYER_3PG']]
y = y['PLAYER_PTS']

X = (X - X.mean()) / X.std()
X.drop(columns=['OFF_POST_TOUCH_PTS', 'OFF_POST_TOUCH_FG_PCT', 'OFF_PTS_PER_100'], inplace=True)

y_pred = model.predict(X)

# Evaluate error
score = model.evaluate(X, y)
mae = mean_absolute_error(y, y_pred)
print('RMSE: ' + str(sqrt(score[0])))
print('MAE: ' + str(mae))

ValueError: in user code:

    File "C:\Users\Darrick\anaconda3\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Darrick\anaconda3\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Darrick\anaconda3\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Darrick\anaconda3\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "C:\Users\Darrick\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Darrick\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 263, in assert_input_compatibility
        raise ValueError(f'Input {

In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

ModuleNotFoundError: No module named 'tensorflow.python.distribute.merge_call_interim'

In [32]:
!pip uninstall tensorflow

^C


In [11]:
!pip uninstall tensorflow

^C
